In [24]:
import numpy as np

coords = np.load("coordinates.npy")

print("Loaded shape:", coords.shape)
print("Sample data point:", coords[0, :5])
print("Sample data point:", coords[0, :5][:,2])

Loaded shape: (406, 20, 3)
Sample data point: [[1.00000000e-01 1.00000000e-01 1.47135305e-07]
 [9.00000000e-01 1.00000000e-01 1.08432184e-02]
 [1.00000000e-01 9.00000000e-01 1.73540304e-01]
 [9.00000000e-01 9.00000000e-01 1.71525839e-01]
 [1.28000000e-01 7.02166656e-01 2.80641319e-01]]
Sample data point: [1.47135305e-07 1.08432184e-02 1.73540304e-01 1.71525839e-01
 2.80641319e-01]


## Expected improvement (EI)

In [ ]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

In [ ]:
optimization_step = 500 
learning_rate = 5e-3
prior_mean = 0.5
prior_std = 0.5 
lin_x = np.linspace(0, 1, 101)
lin_y = np.linspace(0, 1, 101)
lin_xx, lin_yy = np.meshgrid(lin_x, lin_y)
lin_points = np.column_stack([lin_xx.ravel(), lin_yy.ravel()])
log_like_values = []  # Liste til at gemme log-likelihood værdier
log_like_so_far = [] # Liste til at gemme log-likelihood værdier for hver iteration

for trial in coords[:3]:
    print(trial)
    for i,point in enumerate(trial[4:]):
        i = i+4
        x, y, z = point 

        lengthscale, output_var, noise_var = optimize_GP_hyperparams(trial[:i, :2], trial[:i, 2], optimization_step, learning_rate, prior_mean, prior_std) #opdater med den rigtige Gaussian process funktion
        mu, cov = fit_predictive_GP(trial[:i, :2], trial[:i, 2], lin_points, lengthscale, output_var, noise_var) #opdater med den rigtige Gaussian process funktion


        std = np.sqrt(np.diag(cov))
        xi = 0.1 # prøv forskellige værdier
        current_best = np.max(trial[:i+1, 2])

        aq_vals = expected_improvement(current_best, mu, std, xi) # kør med den relavante aquisition funktion

        beta = 5 # prøv forskellige værdier
        sm = softmax(beta * aq_vals)
        sm.reshape(lin_xx.shape) # reshape så det passer med lin_xx og lin_yy
        log_like = np.log(sm[round(x*100)][round(y*100)] + 1e-8) # nok ikke sådan det er indexed
        log_like_so_far.append(log_like)  # Gem log-likelihood værdien
        
    log_like_values.append(log_like_so_far)  # Gem log-likelihood værdierne for denne trial
    log_like_so_far = []  # Nulstil listen for næste trial




ModuleNotFoundError: No module named 'BayesianOptimization'

In [33]:
lin_x = np.linspace(0, 1, 101)
lin_y = np.linspace(0, 1, 101)
lin_xx, lin_yy = np.meshgrid(lin_x, lin_y)
lin_points = np.column_stack([lin_xx.ravel(), lin_yy.ravel()])
print("lin_points shape:", lin_points)

lin_points shape: [[0.   0.  ]
 [0.01 0.  ]
 [0.02 0.  ]
 ...
 [0.98 1.  ]
 [0.99 1.  ]
 [1.   1.  ]]
